# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#512 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846868031649                   -0.70    4.0
  2   -7.852326166110       -2.26       -1.53    1.0
  3   -7.852646089291       -3.49       -2.52    3.0
  4   -7.852646678807       -6.23       -3.34    2.8
  5   -7.852646686117       -8.14       -4.81    1.0
  6   -7.852646686727       -9.22       -5.27    4.5
  7   -7.852646686730      -11.51       -6.03    1.5
  8   -7.852646686730      -12.68       -7.42    2.0
  9   -7.852646686730   +  -15.05       -7.66    2.8
 10   -7.852646686730      -14.75       -8.46    2.0
 11   -7.852646686730      -15.05       -9.48    2.2
 12   -7.852646686730   +  -15.05      -10.59    2.2
 13   -7.852646686730      -14.75      -11.50    2.5
 14   -7.852646686730   +  -14.27      -11.89    2.2
 15   -7.852646686730      -14.75      -13.14    2.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846828492394                   -0.70           4.2
  2   -7.852527926979       -2.24       -1.62   0.80    5.0
  3   -7.852609755363       -4.09       -2.79   0.80    5.2
  4   -7.852646568331       -4.43       -3.59   0.80    7.2
  5   -7.852646683616       -6.94       -4.50   0.80    8.8
  6   -7.852646686668       -8.52       -5.13   0.80   10.5
  7   -7.852646686725      -10.25       -6.23   0.80   11.5
  8   -7.852646686730      -11.31       -7.31   0.80   14.0
  9   -7.852646686730      -13.65       -7.74   0.80   16.0
 10   -7.852646686730   +    -Inf       -9.82   0.80   16.5
 11   -7.852646686730   +  -15.05       -9.74   0.80   20.2
 12   -7.852646686730      -14.57      -10.50   0.80   19.2
 13   -7.852646686730   +    -Inf      -11.38   0.80   21.5
 14   -7.852646686730   +    -Inf      -12.84   0.80   23.5


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.362229e+01     3.279555e+00
 * time: 0.44084715843200684
     1     1.075045e+00     1.665733e+00
 * time: 0.6498169898986816
     2    -1.526241e+00     1.945238e+00
 * time: 0.6732602119445801
     3    -3.624470e+00     1.685818e+00
 * time: 0.784710168838501
     4    -4.720519e+00     1.581546e+00
 * time: 0.8184480667114258
     5    -6.492035e+00     1.003409e+00
 * time: 0.852135181427002
     6    -7.284609e+00     6.006758e-01
 * time: 0.8858370780944824
     7    -7.572279e+00     4.539123e-01
 * time: 0.9091131687164307
     8    -7.728246e+00     1.097130e-01
 * time: 0.9324140548706055
     9    -7.783688e+00     1.130389e-01
 * time: 0.9555699825286865
    10    -7.819041e+00     5.870885e-02
 * time: 0.978748083114624
    11    -7.833602e+00     6.203044e-02
 * time: 1.002216100692749
    12    -7.843596e+00     4.730354e-02
 * time: 1.0256540775299072
    13    -7.846127e+00     4.766830e-02
 * time: 1.0489401817321

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846811310818                   -0.70    4.0
  2   -7.852315629204       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686676                   -2.54
  2   -7.852646686730      -10.27       -5.76
  3   -7.852646686730   +  -14.75      -11.90
  4   -7.852646686730   +    -Inf      -15.88
|ρ_newton - ρ_scf|  = 5.972785112235395e-14
|ρ_newton - ρ_scfv| = 1.0243162092732616e-12
|ρ_newton - ρ_dm|   = 7.406657148409516e-10
